### Imports

In [1]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Sequential, load_model
from keras.layers.recurrent import GRU
from keras.layers import Dense, LSTM, Bidirectional

from encrypt import *
from tools import *
import string

Using TensorFlow backend.


### Configurations

In [2]:
letters = string.ascii_uppercase
encrypt = shifting_cipher
samples_count = 100000
text_length = 5

set_characters(letters)
model_path = f"{encrypt.__name__}_{len(letters)}x{text_length}_best_model.h5"

### Generating and Preparing Data

In [3]:
text = generate_text(text_length, samples_count)

train_cipher = to_vec(map(encrypt, text))
train_text = to_vec(text)

### Building and Training the Model

In [4]:
model = Sequential()
model.add(Bidirectional(GRU(128, return_sequences=True), input_shape=train_cipher.shape[1:]))
model.add(Dense(1))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_1 (Bidirection (None, 5, 256)            99840     
_________________________________________________________________
dense_1 (Dense)              (None, 5, 1)              257       
Total params: 100,097
Trainable params: 100,097
Non-trainable params: 0
_________________________________________________________________


In [5]:
callbacks = [
    EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=3, verbose=1, mode='min'),
    ModelCheckpoint(model_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
]
model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
model.fit(train_cipher, train_text, validation_split=.1, batch_size=10, epochs=5, callbacks=callbacks)

Train on 90000 samples, validate on 10000 samples
Epoch 1/5
90000/90000 [==============================] - 307s 3ms/step - loss: 36.7157 - acc: 0.3281 - val_loss: 8.8477 - val_acc: 0.3159

Epoch 00001: val_acc improved from -inf to 0.31594, saving model to shifting_cipher_26x5_best_model.h5
Epoch 2/5
90000/90000 [==============================] - 56s 624us/step - loss: 1.6294 - acc: 0.7774 - val_loss: 0.0379 - val_acc: 0.9952

Epoch 00002: val_acc improved from 0.31594 to 0.99520, saving model to shifting_cipher_26x5_best_model.h5
Epoch 3/5
90000/90000 [==============================] - 54s 595us/step - loss: 0.0275 - acc: 0.9926 - val_loss: 0.0260 - val_acc: 0.9996

Epoch 00003: val_acc improved from 0.99520 to 0.99958, saving model to shifting_cipher_26x5_best_model.h5
Epoch 4/5
90000/90000 [==============================] - 55s 610us/step - loss: 0.0209 - acc: 0.9967 - val_loss: 0.0091 - val_acc: 1.0000

Epoch 00004: val_acc improved from 0.99958 to 1.00000, saving model to shifting

### Predicting and Evaluating the Model

In [6]:
model = load_model(model_path)

In [7]:
test_cyhper = 'BCDEA'
prediction = to_txt(model.predict(to_vec(test_cyhper)))

print('Model predicted that original text is:', prediction)
print('Test cipher vs encrypted prediction:', test_cyhper, 'vs', encrypt(prediction))

Model predicted that original text is: ABCDE
Test cipher vs encrypted prediction: BCDEA vs BCDEA
